In [44]:
import numpy as np
import pandas as pd
import warnings
from lc_classifier.features import CustomHierarchicalExtractor
warnings.filterwarnings('ignore')

# Reading the sample data

To calculate the model features we will need:
- Detections: Alerts from the ZTF stream, with 5 $\sigma$ difference between the template and the current image.
- Non Detections: Previous measurements of the object but with less than 5 $\sigma$ magnitudes.
- Object Information: Aggregated information by ALeRCE.

In [54]:
detections = pd.read_parquet('data/detections_sample.parquet')
non_detections = pd.read_parquet('data/non_detections_sample.parquet')
object_information = pd.read_parquet('data/object_info_sample.parquet')

In [55]:
detections.head()

,fid,isdiffpos,sigmapsf,field,fwhm,dec,magpsf,rcid,ra,sky,...,sgscore3,neargaia,sgscore1,mjd,corrected,magpsf_corr,sigmapsf_corr,sigmapsf_corr_ext,dubious,has_stamp
oid,,,,,,,,,,,,,,,,,,,,,
ZTF17aaaemke,1,-1.0,0.098972,396,2.910000,0.802001,17.957479,61,358.760453,-0.130846,...,0.441208,0.061395,0.99875,58700.461042,True,16.478576,0.008541,0.025349,False,True
ZTF17aaaemke,1,-1.0,0.075092,396,3.850000,0.801930,17.806704,61,358.760434,0.125953,...,0.441208,0.274704,0.99875,58754.290857,True,16.520813,100.000000,0.022974,False,True
ZTF17aaaemke,1,-1.0,0.093225,396,3.160000,0.801997,17.616571,61,358.760472,0.457313,...,0.441208,0.005324,0.99875,58763.377998,True,16.586325,0.024660,0.036094,False,True
ZTF17aaaemke,1,-1.0,0.146396,396,1.544062,0.801997,18.686962,61,358.760405,-0.294222,...,0.441208,0.233289,0.99875,58718.413889,True,16.350400,100.000000,0.017018,False,True
ZTF17aaaemke,1,-1.0,0.091417,396,3.670000,0.801892,17.775181,61,358.760452,0.110218,...,0.441208,0.386295,0.99875,58372.401910,True,16.530643,0.014738,0.029054,False,True


In [56]:
non_detections.head()

,parent_candid,jd,fid,pid,diffmaglim,pdiffimfilename,programpi,programid,candid,isdiffpos,...,ranr,decnr,scorr,magzpsci,magzpsciunc,magzpscirms,clrcoeff,clrcounc,rbversion,mjd
oid,,,,,,,,,,,,,,,,,,,,,
ZTF17aaaemke,618401906115010002,2.458353e+06,1,598377030615,20.065100,/ztf/archive/sci/2018/0822/376991/ztf_20180822...,Kulkarni,1,NaN,None,...,NaN,NaN,NaN,0.0,0.0,0.0,0.0,0.0,t12_f5_c3,58352.377037
ZTF17aaaemke,636353206115010001,2.458361e+06,1,606403590615,19.109699,/ztf/archive/sci/2018/0830/403600/ztf_20180830...,Kulkarni,1,NaN,None,...,NaN,NaN,NaN,0.0,0.0,0.0,0.0,0.0,t12_f5_c3,58360.403600
ZTF17aaaemke,618401906115010002,2.458364e+06,1,609312890615,20.057800,/ztf/archive/sci/2018/0902/312859/ztf_20180902...,Kulkarni,1,NaN,None,...,NaN,NaN,NaN,0.0,0.0,0.0,0.0,0.0,t12_f5_c3,58363.312894
ZTF17aaaemke,618401906115010002,2.458367e+06,1,612400500615,20.791800,/ztf/archive/sci/2018/0905/400509/ztf_20180905...,Kulkarni,1,NaN,None,...,NaN,NaN,NaN,0.0,0.0,0.0,0.0,0.0,t12_f5_c3,58366.400509
ZTF17aaaemke,618401906115010002,2.458370e+06,1,615325740615,20.812700,/ztf/archive/sci/2018/0908/325729/ztf_20180908...,Kulkarni,1,NaN,None,...,NaN,NaN,NaN,0.0,0.0,0.0,0.0,0.0,t12_f5_c3,58369.325741


In [57]:
object_information.head()

,ndethist,ncovhist,mjdstarthist,mjdendhist,meanra,meandec,sigmara,sigmadec,firstmjd,lastmjd,...,nearZTF,nearPS1,stellar,corrected,ndet,ndubious,g-r_max,g-r_max_corr,g-r_mean,g-r_mean_corr
oid,,,,,,,,,,,,,,,,,,,,,
ZTF17aaaemke,135.0,670.0,58302.481944,59107.311562,358.760440,0.801990,0.000086,0.000060,58372.401910,59107.311562,...,True,False,True,True,88,0,1.209530,0.828800,0.531433,0.952658
ZTF17aaafyya,968.0,1471.0,58101.201285,59107.366215,17.787271,61.129036,0.000128,0.000066,58314.467755,59107.366215,...,True,True,True,True,375,3,0.757556,0.824260,0.635323,0.916122
ZTF17aaageae,280.0,783.0,58351.390069,59107.310625,8.686962,0.905769,0.000107,0.000067,58355.371991,59107.310625,...,True,True,True,True,217,0,0.168999,-0.030895,0.063114,0.207873
ZTF17aaaivsr,232.0,669.0,58101.410752,58981.216250,129.397864,9.654056,0.000060,0.000064,58375.516100,58981.216250,...,True,True,True,True,79,0,0.663141,0.630097,0.823536,0.559042
ZTF17aaaizej,332.0,428.0,58101.419444,58987.201377,141.224907,4.263769,0.000071,0.000050,58472.420324,58987.201377,...,True,True,True,True,100,0,-0.603085,-0.105530,0.113668,0.197381


# Calculating features

In [58]:
# The extractor used is CustomHirarchicalExtractor
features_computer = CustomHierarchicalExtractor()

In [59]:
# We use the compute_features method
features = features_computer.compute_features(
    detections = detections,
    non_detections = non_detections,
    objects = object_information)

In [61]:
# The result is a dataframe with the features
features

,Amplitude_1,Amplitude_2,AndersonDarling_1,AndersonDarling_2,Autocor_length_1,Autocor_length_2,Beyond1Std_1,Beyond1Std_2,Con_1,Con_2,...,n_non_det_after_fid_1,n_non_det_after_fid_2,n_non_det_before_fid_1,n_non_det_before_fid_2,n_pos_1,n_pos_2,positive_fraction_1,positive_fraction_2,rb,sgscore1
oid,,,,,,,,,,,,,,,,,,,,,
ZTF17aaaemke,0.319945,0.277661,0.999998,0.999997,1.0,1.0,0.450000,0.363636,0.026316,0.0,...,105,103,7,7,5,12,0.125000,0.363636,0.880000,0.998750
ZTF17aaafyya,0.373089,0.347666,1.000000,1.000000,1.0,1.0,0.404580,0.391753,0.000000,0.0,...,89,82,1,1,52,43,0.396947,0.443299,0.787857,0.996250
ZTF17aaageae,0.219709,0.338611,1.000000,1.000000,1.0,1.0,0.340000,0.105263,0.000000,0.0,...,133,177,5,1,28,16,0.560000,0.421053,0.749286,1.000000
ZTF17aaaivsr,0.343258,0.305270,0.999999,0.821926,1.0,1.0,0.371429,0.400000,0.000000,0.0,...,24,71,2,0,13,4,0.371429,0.160000,0.791667,0.992012
ZTF17aaaizej,0.675934,0.404425,1.000000,1.000000,1.0,1.0,0.325000,0.311111,0.000000,0.0,...,8,16,1,1,14,8,0.350000,0.177778,0.843333,1.000000
ZTF17aaajizt,NaN,0.462060,NaN,0.993290,NaN,1.0,NaN,0.260870,NaN,0.0,...,NaN,18,NaN,1,NaN,10,NaN,0.434783,0.763333,0.987768
ZTF17aaajlul,0.407220,0.318167,1.000000,1.000000,1.0,1.0,0.425000,0.408163,0.000000,0.0,...,28,32,0,0,10,8,0.250000,0.163265,0.817143,1.000000
ZTF17aaakstv,0.157715,0.294221,0.987377,0.999638,2.0,1.0,0.375000,0.416667,0.000000,0.0,...,32,50,1,1,4,4,0.500000,0.333333,0.686905,0.998750
ZTF17aaanxog,0.578554,0.433526,0.998814,0.978531,1.0,1.0,0.388889,0.384615,0.000000,0.0,...,61,59,2,3,7,2,0.388889,0.153846,0.791429,0.996875


To this features we add some other features, more specific WISE colors.